In [ ]:
from cling import cling, bash

**Listing 10.1**

Caption: Implementation of the Taylor series of the natural logarithm using parallel algorithms.

In [ ]:
%%cling
#include <vector>
#include <cmath>
#include <numeric>
#include <algorithm>
#include <hpx/hpx.hpp>
//-----------------------------------------------------------------
using hpx::for_each, hpx::execution::par, hpx::reduce;

const int n = 400;
const double x = 0.1;
std::vector<double> parts(n);
std::iota(parts.begin(), parts.end(), 1);

for_each(par, parts.begin(), parts.end(), [](double& e) { 
    e = std::pow(-1.0, e + 1) * std::pow(x, e) / (e);
});

double result = reduce(par, parts.begin(), parts.end(), 0.); 


**Listing 10.2**

Caption: Implementation of the Taylor series of the natural logarithm using hpx::experimental::for_loop.

In [ ]:
%%cling
#include <array>
#include <hpx/parallel/algorithm.hpp>
//-----------------------------------------------------------------
std::array<int, 100000> a;

hpx::experimental::for_loop(
        hpx::execution::par,
        0, a.size(), [&](size_t i) { 
	a[i] = i * i ;
});


**Listing 10.3**

Caption: Implementation of the Taylor series of the natural logarithm using HPX's parallel algorithms.

In [ ]:
%%cling
#include <array>
#include <cmath>
#include <hpx/hpx.hpp>
#include <hpx/numeric.hpp>
#include <hpx/parallel/algorithm.hpp>
//-----------------------------------------------------------------
std::array<int, 100000> a;

hpx::future<void> f =
  hpx::experimental::for_loop(
    hpx::execution::par(hpx::execution::task), 
    0,
    a.size(),
    [](size_t i) {
      a[i] = i * i ;
    });

// Use the future for synchronization
f.then([](auto&& f) { 
	std::cout << hpx::reduce(std::begin(a), std::end(a), 0) << std::endl;
});


**Listing 10.4**

Caption: Using SIMD within HPX's parallel algorithms via execution policies.

In [ ]:
%%writefile Listing_10_4.cpp
#include <array>
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <hpx/numeric.hpp>
#include <hpx/parallel/algorithm.hpp>
#include <hpx/parallel/datapar.hpp>
//-----------------------------------------------------------------
int main() {
  std::array<int, 100000> a;

  std::cout << hpx::reduce(hpx::execution::simd, std::begin(a), std::end(a), 0) << std::endl; 

  std::cout << hpx::reduce(hpx::execution::par_simd, std::begin(a), std::end(a), 0) << std::endl; 
}


In [ ]:
%%bash
hpxcxx -I . --exe=Listing_10_4.exe Listing_10_4.cpp
hpxrun.py -l 3 -t 1 ./Listing_10_4.exe

**Listing 10.5**

Caption: Using chunk sizes within HPX's parallel algorithms.

In [ ]:
%%cling
#include <array>
#include <hpx/hpx.hpp>
#include <hpx/parallel/algorithm.hpp>
//-----------------------------------------------------------------
std::array<int, 100000> a;

// Static chunk size
hpx::execution::static_chunk_size scs(10); 
hpx::experimental::for_loop(hpx::execution::par.with(scs), 0, a.size(), 
                [&](size_t i) { a[i] = i * i; });

// Dynamic chunk size
hpx::execution::dynamic_chunk_size dcs(10); 
hpx::experimental::for_loop(hpx::execution::par.with(dcs), 0, a.size(),
                [&](size_t i) { a[i] = i * i; });


**Listing 10.6**

Caption: Example for the computation of the fractal sets using HPX's parallel algorithms.

In [ ]:
%%cling
#include <hpx/parallel/algorithm.hpp>

#include <pbm.hpp>
#include <config.hpp>
#include <kernel.hpp>
//-----------------------------------------------------------------
// Definition of utility
PBM pbm = PBM(size_x, size_y);

hpx::experimental::for_loop(hpx::execution::par,0,size_x, [&](size_t i) {


    complex c =
        complex(0, 4) * complex(i, 0) / complex(size_x, 0) - complex(0, 2);

    for (size_t j = 0; j < size_y; j++) {
      // Get the number of iterations
      int value = compute_pixel(c + 4.0 * j / size_y - 2.0);
      // Convert the value to RGB color space
      std::tuple<size_t, size_t, size_t> color = get_rgb(value);
      // Set the pixel color
      pbm(i, j) = make_color(std::get<0>(color),
                             std::get<1>(color),
                             std::get<2>(color));
    }
  });

  // Save the image
  pbm.save("image_parallel_" + type + ".pbm");


**Listing 10.7**

Caption: Example for the computation of the fractal sets using OpenMP.

In [ ]:
%%cling
#include <chrono>

#include <config.hpp>
#include <kernel.hpp>
#include <pbm.hpp>
//-----------------------------------------------------------------
// Definition of utility
PBM pbm = PBM(size_x, size_y);

#pragma omp parallel for
  for (size_t i = 0; i < size_x; i++) {
    complex c =
        complex(0, 4) * complex(i, 0) / complex(size_x, 0) - complex(0, 2);

    for (size_t j = 0; j < size_y; j++) {
      // Get the number of iterations
      int value = compute_pixel(c + 4.0 * j / size_y - 2.0);
      // Convert the smoothened value to RGB color space
      std::tuple<size_t, size_t, size_t> color = get_rgb(value);
      // Set the pixel color
      pbm(i, j) = make_color(std::get<0>(color), std::get<1>(color),
                             std::get<2>(color));
    }
  }

// Save the image
pbm.save("image_parallel_" + type + ".pbm");
